In [1]:
!pip install --upgrade pytorch_tabnet


In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.utils import to_categorical
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.callbacks import EarlyStopping
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier
from pytorch_tabnet.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


class NeuralNetwork(BaseEstimator, TransformerMixin):
    
    def __init__(self, conv1d_filters=64, conv1d_kernel_size=3, conv2d_filters=16, 
                 conv2d_kernel_size=(3, 3), tabnet_feature_dim=32, tabnet_num_attention_heads=4,
                 tabnet_num_decision_steps=4, tabnet_virtual_batch_size=64, tabnet_batch_size=1024,
                 tabnet_dropout=0.2, conv1d_epochs=50, conv1d_patience=10, conv2d_epochs=50,
                 conv2d_patience=10, tabnet_epochs=50, tabnet_patience=10, num_classes=2):
        self.conv1d_filters = conv1d_filters
        self.conv1d_kernel_size = conv1d_kernel_size
        self.conv2d_filters = conv2d_filters
        self.conv2d_kernel_size = conv2d_kernel_size
        self.tabnet_feature_dim = tabnet_feature_dim
        self.tabnet_num_attention_heads = tabnet_num_attention_heads
        self.tabnet_num_decision_steps = tabnet_num_decision_steps
        self.tabnet_virtual_batch_size = tabnet_virtual_batch_size
        self.tabnet_batch_size = tabnet_batch_size
        self.tabnet_dropout = tabnet_dropout
        self.conv1d_epochs = conv1d_epochs
        self.conv1d_patience = conv1d_patience
        self.conv2d_epochs = conv2d_epochs
        self.conv2d_patience = conv2d_patience
        self.tabnet_epochs = tabnet_epochs
        self.tabnet_patience = tabnet_patience
        self.num_classes = num_classes
        self.keras_model = None
        self.pytorch_model = None
        self.tabnet_model = None
        
        
    def fit(self, X, y):
        # 1D Convolutional Neural Network
        self.keras_model = Sequential()
        self.keras_model.add(Conv1D(self.conv1d_filters, self.conv1d_kernel_size, activation='relu', input_shape=(X.shape[1], 1)))
        self.keras_model.add(MaxPooling1D(pool_size=2))
        self.keras_model.add(Conv1D(self.conv1d_filters*2, self.conv1d_kernel_size, activation='relu'))
        self.keras_model.add(MaxPooling1D(pool_size=2))
        self.keras_model.add(Flatten())
        self.keras_model.add(Dense(self.num_classes, activation='softmax'))
        self.keras_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        X_reshape = np.reshape(X, (X.shape[0], X.shape[1], 1))
        y_categorical = to_categorical(y)
        self.keras_model.fit(X_reshape, y_categorical, epochs=self.conv1d_epochs, 
                             validation_split=0.2, callbacks=[EarlyStopping(patience=self.conv1d_patience)])
        
        # 2D Convolutional Neural Network
        self.pytorch_model = CNN2D(X.shape[1], self.conv2d_filters, self.num_classes, kernel_size=self.conv2d_kernel_size)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.pytorch_model.parameters())
        early_stop = EarlyStopping(patience=self.conv2d_patience)
        X_reshape = np.reshape(X, (X.shape[0], 1, X.shape[1], 1))
        X_train, X_val, y_train, y_val = train_test_split(X_reshape, y, test_size=0.2, random_state=42)
        train_dataset = CustomDataset(X_train, y_train)
        val_dataset = CustomDataset(X_val, y_val)
        train_loader = DataLoader(train_dataset, batch_size=64)
        val_loader = DataLoader(val_dataset, batch_size=64)
        train_losses, val_losses = train_model(self.pytorch_model, train_loader, val_loader, criterion, optimizer, 
                                                self.conv2d_epochs, early_stop)
        
        # TabNet Classifier
        self.tabnet_model = TabNetClassifier(
            n_d=self.tabnet_feature_dim, 
            n_a=self.tabnet_num_attention_heads,
            n_steps=self.tabnet_num_decision_steps,
            gamma=1.5,
            n_independent=2,
            n_shared=2,
            seed=42,
            momentum=0.02,
            clip_value=2.,
            lambda_sparse=0.,
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=2e-2),
            mask_type=TabNetMaskType.entmax,
            scheduler_params=dict(mode="min",
                                  patience=self.tabnet_patience,
                                  min_lr=1e-5,
                                  factor=0.5,),
            scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
            verbose=0
        )
                                      
        # fitting the tabnet model
        self.tabnet_model.fit(X_train=X_train, y_train=y_train, X_valid=X_val, y_valid=y_val,
                              patience=self.tabnet_patience, batch_size=self.tabnet_batch_size, 
                              virtual_batch_size=self.tabnet_virtual_batch_size, 
                              num_workers=self.tabnet_num_workers, drop_last=False)
        
        print("Training completed.")
        
        
        
    def predict(self, X):
        """
        Predict the output for input X
        
        Args:
        - X: input data
        
        Returns:
        - y_pred: predicted labels
        """
        X_reshape = np.reshape(X, (X.shape[0], 1, X.shape[1], 1))
        y_pred_cnn1d = self.keras_model.predict(X_reshape)
        y_pred_cnn2d = self.pytorch_model.predict(torch.tensor(X_reshape).float())
        y_pred_tabnet = self.tabnet_model.predict(X)
        y_pred = np.mean([y_pred_cnn1d, y_pred_cnn2d, y_pred_tabnet], axis=0)
        return np.round(y_pred)
    
    # create a pipeline object
    pipeline = Pipeline([('scaler', StandardScaler()), ('nn', NeuralNetwork())])

    
    def evaluate_pipeline(X_test, y_test, pipeline):
     """
        Evaluate the pipeline on the test set and display performance metrics
    
        Args:
        - X_test (numpy array): test set input
        - y_test (numpy array): test set target
        - pipeline (Pipeline): trained pipeline
    
        Returns:
        - None
    """
    # make predictions using the pipeline
    y_pred = pipeline.predict(X_test)
    
    # calculate performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # plot confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    
    # display performance metrics
    print(f'Accuracy: {accuracy:.3f}')
    print(f'Precision: {precision:.3f}')
    print(f'Recall: {recall:.3f}')
    print(f'F1-score: {f1:.3f}')





2023-04-27 20:27:14.482106: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 20:27:14.632965: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-27 20:27:14.632987: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-27 20:27:15.441752: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

NameError: name 'pipeline' is not defined


The `predict()` method takes input `X`, reshapes it, and predicts the output labels for each of the three models. The final prediction is the average of the predictions from each model. The output is rounded to obtain binary labels.

Now we will define a `Pipeline` class that will allow us to run the entire data preprocessing and model training process in a single step. The `Pipeline` class will have the following methods:
- `__init__()`: constructor method that initializes the parameters and the models
- `fit()`: fits the model on the input data
- `predict()`: predicts the output for input data



In [ ]:
import xarray as xr
import json
from typing import Union
from pathlib import Path
import glob



def to_path(p: Union[str, Path]) -> Path:
    return p if isinstance(p, Path) else Path(p)

def find_records(path: str):
    search_path: str = f"{path}/**/signals/"
    all_paths = list(map(lambda x: str(to_path(x).parent), glob.glob(search_path, recursive=True)))
    return all_paths

class RecordReader():
    def __init__(self, path: Union[str, Path]):
        self.path = to_path(path)

    def load_signal(self, sig_name):
        return xr.open_zarr(self.path / "signals" / sig_name / "dataset")

    def load_signal_meta(self, sig_name):
        with open(self.path / "signals" / sig_name / "meta.json", "r") as meta:
            return json.load(meta)
    
    def load_metadata(self):
        with open(self.path / "meta.json", "r") as meta:
            return json.load(meta)

    def load_crf_metadata(self):
        with open(self.path / "crf.json", "r") as meta:
            return json.load(meta)

In [ ]:
records = find_records(("./"))
print(records, flush=True)

In [ ]:
data = {}
for r in records:
    reader = RecordReader(r)
    metadata = reader.load_metadata()
    scg_metadata = reader.load_signal_meta('scg-k')
    rsp_metadata = reader.load_signal_meta('rsp')
    crf_data = reader.load_crf_metadata()
    
    value = {
            'age': metadata['subject']['age']['value'],
            'sex' : metadata['subject']['sex'],
            'weight': metadata['subject']['weight']['value'],
            'height' : metadata['subject']['height']['value'],
            'subject_id' : crf_data['subject_id'],
            'study_id' : crf_data['study_id'],
            'hf_type' : crf_data['hf_type'],
            'sample_rate_scgk' : scg_metadata['sample_rate'],
            'nrg_lin_scgk' : reader.load_signal("scg-k").nrg.sel(motion="lin").to_pandas(),
            'nrg_rot_scgk' : reader.load_signal("scg-k").nrg.sel(motion="rot").to_pandas(),
            'pwr_lin_scgk': reader.load_signal("scg-k").pwr.sel(motion="lin").to_pandas(),
            'pwr_rot_scgk': reader.load_signal("scg-k").pwr.sel(motion="rot").to_pandas(),
            'sample_rate_rsp' : rsp_metadata['sample_rate'],
            'rsp': reader.load_signal("rsp").signal.to_pandas()
            }
    data[metadata['id']] = value

In [ ]:
df = pd.DataFrame.from_dict(data, orient='index')

In [ ]:
import numpy as np
from scipy.stats import skew, kurtosis
import pywt

def calculate_features(ts):
    mean = np.mean(ts)
    std = np.std(ts)
    median = np.median(ts)
    minimum = np.min(ts)
    maximum = np.max(ts)
    skewness = skew(ts)
    kurt = kurtosis(ts)
    rms = np.sqrt(np.mean(np.square(ts)))
    zero_crossings = np.sum(np.diff(np.sign(ts)) != 0)
    
    # Wavelet transformation
    wavelet = 'db4'
    coeffs = pywt.wavedec(ts, wavelet, level=4)
    
    # Calculate wavelet features
    wavelet_mean = np.mean(np.concatenate(coeffs))
    wavelet_std = np.std(np.concatenate(coeffs))
    wavelet_energy = np.sum(np.square(np.concatenate(coeffs)))
    
    return [mean, std, median, minimum, maximum, skewness, kurt, rms, zero_crossings, wavelet_mean, wavelet_std, wavelet_energy]

# Compute time series features for each subject
for subject_id, value in data.items():
    for feature in ['nrg_lin_scgk', 'nrg_rot_scgk', 'pwr_lin_scgk', 'pwr_rot_scgk', 'rsp']:
        ts = value[feature].values
        features = calculate_features(ts)
        
        # Store the computed features
        value[f"{feature}_mean"] = features[0]
        value[f"{feature}_std"] = features[1]
        value[f"{feature}_median"] = features[2]
        value[f"{feature}_min"] = features[3]
        value[f"{feature}_max"] = features[4]
        value[f"{feature}_skew"] = features[5]
        value[f"{feature}_kurt"] = features[6]
        value[f"{feature}_rms"] = features[7]
        value[f"{feature}_zero_crossings"] = features[8]
        value[f"{feature}_wavelet_mean"] = features[9]
        value[f"{feature}_wavelet_std"] = features[10]
        value[f"{feature}_wavelet_energy"] = features[11]

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index')

In [ ]:
df.dtypes

In [ ]:
# Checking target
df.hf_type.value_counts()

In [ ]:
# Dropping the hf_type = UNKNOWN
df = df[df["hf_type"]!="UNKNOWN"]

In [ ]:
# Check counts again
df.hf_type.value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Encode the hf_type column as integer labels
encoder = LabelEncoder()
df['hf_type'] = encoder.fit_transform(df['hf_type'])

In [ ]:
# Creating a function to code HFpEF and HFmEF into one category, NoHF second category, and HFrEF third
def convert(df):
    if df["hf_type"]==3:
        return 0
    elif df["hf_type"]==2:
        return 2
    else:
        return 1

In [ ]:
df["hf_type"] = df.apply(lambda df: convert(df), axis=1)

In [ ]:
from scipy.signal import periodogram
from scipy.stats import entropy

def spectral_entropy(pxx):
    psd_norm = pxx / np.sum(pxx)
    return entropy(psd_norm)

# Initialize a list to store the feature data
feature_data = []

# Iterate through the data dictionary
for key, value in data.items():
    # Calculate the periodogram for each time series
    freq_nrg_lin, pxx_nrg_lin = periodogram(value['nrg_lin_scgk'])
    freq_nrg_rot, pxx_nrg_rot = periodogram(value['nrg_rot_scgk'])
    freq_pwr_lin, pxx_pwr_lin = periodogram(value['pwr_lin_scgk'])
    freq_pwr_rot, pxx_pwr_rot = periodogram(value['pwr_rot_scgk'])
    freq_rsp, pxx_rsp = periodogram(value['rsp'])

    # Calculate the frequency-domain features for each time series
    features = {
        'record_id': key,
        'nrg_lin_mean_freq': np.mean(freq_nrg_lin),
        'nrg_lin_median_freq': np.median(freq_nrg_lin),
        'nrg_lin_peak_freq': freq_nrg_lin[np.argmax(pxx_nrg_lin)],
        'nrg_lin_spectral_entropy': spectral_entropy(pxx_nrg_lin),
        'nrg_rot_mean_freq': np.mean(freq_nrg_rot),
        'nrg_rot_median_freq': np.median(freq_nrg_rot),
        'nrg_rot_peak_freq': freq_nrg_rot[np.argmax(pxx_nrg_rot)],
        'nrg_rot_spectral_entropy': spectral_entropy(pxx_nrg_rot),
        'pwr_lin_mean_freq': np.mean(freq_pwr_lin),
        'pwr_lin_median_freq': np.median(freq_pwr_lin),
        'pwr_lin_peak_freq': freq_pwr_lin[np.argmax(pxx_pwr_lin)],
        'pwr_lin_spectral_entropy': spectral_entropy(pxx_pwr_lin),
        'pwr_rot_mean_freq': np.mean(freq_pwr_rot),
        'pwr_rot_median_freq': np.median(freq_pwr_rot),
        'pwr_rot_peak_freq': freq_pwr_rot[np.argmax(pxx_pwr_rot)],
        'pwr_rot_spectral_entropy': spectral_entropy(pxx_pwr_rot),
        'rsp_mean_freq': np.mean(freq_rsp),
        'rsp_median_freq': np.median(freq_rsp),
        'rsp_peak_freq': freq_rsp[np.argmax(pxx_rsp)],
        'rsp_spectral_entropy': spectral_entropy(pxx_rsp)
    }
    
    # Add the features to the feature_data list
    feature_data.append(features)

# Convert the feature_data list into a DataFrame
features_df = pd.DataFrame(feature_data)

In [ ]:
features_df.head()

In [ ]:
# Set the index of features_df to be the record_id
features_df.set_index('record_id', inplace=True)

In [ ]:
# Join the main DataFrame with the features_df
combined_df = pd.merge(df, features_df, left_index=True, right_index=True)

In [ ]:
combined_df.head()

In [ ]:
combined_df.columns

In [ ]:
print(combined_df.dtypes.head(50))

In [ ]:
combined_df.isna().sum()

In [ ]:
combined_df.shape

In [ ]:
#prepare data for modeling
combined_df.drop(columns=['nrg_lin_scgk', 'nrg_rot_scgk', 'pwr_lin_scgk', 'pwr_rot_scgk', 'rsp'], inplace=True)
skim(combined_df)

In [ ]:
#separate each signal
nrg_signal = combined_df[['nrg_lin_mean_freq', 'nrg_lin_median_freq', 'nrg_lin_peak_freq', 'nrg_lin_spectral_entropy', 'nrg_rot_mean_freq', 'nrg_rot_median_freq', 'nrg_rot_peak_freq', 'nrg_rot_spectral_entropy']]
rot_signal = combined_df[['pwr_lin_mean_freq', 'pwr_lin_median_freq', 'pwr_lin_peak_freq', 'pwr_lin_spectral_entropy', 'pwr_rot_mean_freq', 'pwr_rot_median_freq', 'pwr_rot_peak_freq', 'pwr_rot_spectral_entropy']]
pwr_signal = combined_df[['rsp_mean_freq', 'rsp_median_freq', 'rsp_peak_freq', 'rsp_spectral_entropy']]

In [ ]:
print(nrg_signal.shape)
print(rot_signal.shape)
print(pwr_signal.shape)

In [ ]:
nrg_signal.columns

In [ ]:
#Take a smaller sample to save memory
sample = combined_df.sample(n=100)

In [ ]:
# Split the dataset into X and y
X = sample.drop('hf_type', axis=1)  # Drop the 'target' column to get the feature matrix
y = sample['hf_type']  # Extract the 'target' column to get the target vector


In [ ]:
#fit the data for the pipeline
fit(X, y)

In [ ]:
predict(X)

In [ ]:
evaluate_model(X, y)